In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json

In [2]:
PATH = "../results/REINFORCE/batch0"
ids = range(27)

batch0 = []
for i in ids:
    exp = {'id': i}
    exp.update(json.load(open(f"{PATH}/{i}/args.json", "r")))
    
    evalResults = pickle.load(open(f"{PATH}/{i}/pickle/evalResults.p", "rb"))
    exp['meanReward'] = np.mean(evalResults['rewards'])
    exp['stdReward'] = np.std(evalResults['rewards'])
    batch0.append(exp)
    

In [3]:
PATH = "../results/REINFORCE"
ids = range(1, 66)

batch1 = []
for i in ids:
    exp = {'id': i}
    exp.update(json.load(open(f"{PATH}/{i}/args.json", "r")))
    try:
        evalResults = pickle.load(open(f"{PATH}/{i}/pickle/evalResults.p", "rb"))
        exp['meanReward'] = np.mean(evalResults['rewards'])
        exp['stdReward'] = np.std(evalResults['rewards'])
        batch1.append(exp)
    except:
        print("Something is missing")


Something is missing
Something is missing


In [4]:
experimentDF = pd.DataFrame(batch0 + batch1)

In [5]:
experimentDF

,id,env,nRefs,aCost,a_lr,discount,episodes,max_episode_len,buffer_size,batch_size,update_freq,weightDecay,plots,plot_freq,meanReward,stdReward
0,0,linear-with-ref-v0,1,0.000,0.00001,0.3,1000,1000,5000,128,1,0.000,True,25,-25.154289,1.671577
1,1,linear-with-ref-v0,1,0.000,0.00001,0.3,1000,1000,5000,128,1,0.000,True,25,-82.653905,4.828897
2,2,linear-with-ref-v0,1,0.000,0.00001,0.3,1000,1000,5000,128,1,0.000,True,25,-42.717482,4.010673
3,3,linear-with-ref-v0,1,0.000,0.00001,0.3,1000,1000,5000,128,1,0.001,True,25,-6.624603,1.226016
4,4,linear-with-ref-v0,1,0.000,0.00001,0.3,1000,1000,5000,128,1,0.001,True,25,-103.435361,7.509522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,61,linear-with-ref-v0,2,0.000,0.00001,0.3,500,1000,5000,128,1,0.100,False,10,-79.694196,5.350248
86,62,linear-with-ref-v0,2,0.001,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-30.689710,4.864548
87,63,linear-with-ref-v0,2,0.001,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-120.359742,10.947273
88,64,linear-with-ref-v0,2,0.001,0.00001,0.3,500,1000,5000,128,1,0.001,False,10,-4.357244,1.352987


In [6]:
experimentDF[experimentDF['meanReward'] > -5]

,id,env,nRefs,aCost,a_lr,discount,episodes,max_episode_len,buffer_size,batch_size,update_freq,weightDecay,plots,plot_freq,meanReward,stdReward
15,15,linear-with-ref-v0,1,0.001,0.00001,0.3,1000,1000,5000,128,1,0.001,True,25,-1.267614,0.518078
19,19,linear-with-ref-v0,1,0.001,0.00001,0.3,1000,1000,5000,128,1,0.100,True,25,-0.551864,0.249112
24,24,linear-with-ref-v0,1,0.100,0.00001,0.3,1000,1000,5000,128,1,0.000,True,25,-1.827160,2.568443
29,5,linear-with-ref-v0,1,0.000,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-0.957260,0.316716
39,15,linear-with-ref-v0,1,0.001,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-0.560531,0.494951
45,21,linear-with-ref-v0,1,0.001,0.00001,0.3,500,1000,5000,128,1,0.100,False,10,-4.325819,4.170840
48,24,linear-with-ref-v0,1,0.001,0.00001,0.3,500,1000,5000,128,1,1.000,False,10,-2.808620,0.596733
53,29,linear-with-ref-v0,1,0.100,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-1.820089,0.589298
71,47,linear-with-ref-v0,1,1.000,0.00001,0.3,500,1000,5000,128,1,0.100,False,10,-0.567304,0.137053
75,51,linear-with-ref-v0,2,0.000,0.00001,0.3,500,1000,5000,128,1,0.000,False,10,-1.510521,0.582403


In [7]:
experimentDF.groupby('aCost').mean()

,id,nRefs,a_lr,discount,episodes,max_episode_len,buffer_size,batch_size,update_freq,weightDecay,plots,plot_freq,meanReward,stdReward
aCost,,,,,,,,,,,,,,
0.000,22.342857,1.314286,0.00001,0.3,671.428571,1000.0,5000.0,128.0,1.0,0.254543,0.342857,15.142857,-124.716413,10.014770
0.001,25.357143,1.142857,0.00001,0.3,714.285714,1000.0,5000.0,128.0,1.0,0.236000,0.428571,16.428571,-41.257000,4.407292
0.100,31.000000,1.000000,0.00001,0.3,600.000000,1000.0,5000.0,128.0,1.0,0.220200,0.200000,13.000000,-33.595594,3.168048
1.000,44.500000,1.000000,0.00001,0.3,500.000000,1000.0,5000.0,128.0,1.0,0.275250,0.000000,10.000000,-164.978384,12.601536


In [8]:
experimentDF.groupby('weightDecay').mean()

,id,nRefs,aCost,a_lr,discount,episodes,max_episode_len,buffer_size,batch_size,update_freq,plots,plot_freq,meanReward,stdReward
weightDecay,,,,,,,,,,,,,,
0.000,25.384615,1.192308,0.138769,0.00001,0.3,673.076923,1000.0,5000.0,128.0,1.0,0.346154,15.192308,-80.397151,6.383415
0.001,28.434783,1.217391,0.143826,0.00001,0.3,630.434783,1000.0,5000.0,128.0,1.0,0.260870,13.913043,-46.950355,5.296851
0.100,28.095238,1.142857,0.157429,0.00001,0.3,642.857143,1000.0,5000.0,128.0,1.0,0.285714,14.285714,-193.044485,14.949381
1.000,29.350000,1.100000,0.165300,0.00001,0.3,650.000000,1000.0,5000.0,128.0,1.0,0.300000,14.500000,-38.991334,3.546347


In [9]:
experimentDF.groupby('nRefs').mean()

,id,aCost,a_lr,discount,episodes,max_episode_len,buffer_size,batch_size,update_freq,weightDecay,plots,plot_freq,meanReward,stdReward
nRefs,,,,,,,,,,,,,,
1,21.613333,0.180320,0.00001,0.3,680.0,1000.0,5000.0,128.0,1.0,0.264240,0.36,15.4,-96.576422,7.976399
2,58.000000,0.000267,0.00001,0.3,500.0,1000.0,5000.0,128.0,1.0,0.153667,0.00,10.0,-50.714222,4.962026


In [10]:
PATH = "../results/ACD"
ids = range(24)

acdExp = []
for i in ids:
    exp = {'id': i}
    exp.update(json.load(open(f"{PATH}/{i}/args.json", "r")))
    
    evalResults = pickle.load(open(f"{PATH}/{i}/pickle/evalResults.p", "rb"))
    exp['meanReward'] = np.mean(evalResults['rewards'])
    exp['stdReward'] = np.std(evalResults['rewards'])
    acdExp.append(exp)

acdDF = pd.DataFrame(acdExp)

In [11]:
acdDF

,id,env,cpus,episodes,max_episode_len,nRefs,discount,c_lr,a_lr,batch_size,...,tau,update_freq,aCost,weightDecay,epsilonDecay,exploration_std,plots,plot_freq,meanReward,stdReward
0,0,linear-with-ref-v0,1,500,1000,1,0.0,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-573.502213,6.015824
1,1,linear-with-ref-v0,1,500,1000,1,0.0,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-15.129395,2.656728
2,2,linear-with-ref-v0,1,500,1000,1,0.0,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-19.391742,1.081356
3,3,linear-with-ref-v0,1,500,1000,1,0.3,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-27.862005,2.270990
4,4,linear-with-ref-v0,1,500,1000,1,0.3,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-58.373030,5.551825
5,5,linear-with-ref-v0,1,500,1000,1,0.3,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-15.277938,2.890327
6,6,linear-with-ref-v0,1,500,1000,1,0.7,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-129.051944,6.435204
7,7,linear-with-ref-v0,1,500,1000,1,0.7,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-1818.581153,608.192505
8,8,linear-with-ref-v0,1,500,1000,1,0.7,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-277.855866,15.257610
9,9,linear-with-ref-v0,1,500,1000,1,0.9,0.001,0.00001,128,...,0.005,2,0.0,0.0,3.0,0.5,False,10,-44.587795,1.242653
